# Database overview notebook

> First time use: follow instructions in the README.md file in this directory.

## Setup

In [16]:
from timelink.mhk.utilities import get_engine, get_dbnames, get_connection_string
from sqlalchemy import engine, text, MetaData
print('Available databases:')
print(get_dbnames())
#%load_ext sql
#%config SqlMagic.displaycon=False

Available databases:
['alunos', 'angelica', 'api_tests', 'china', 'china_coimbra', 'dyncoopnet', 'eiras', 'familias', 'glopes', 'ilhavo', 'ilhavo2', 'ilhavo_contrib', 'ilhavo_editor', 'ilhavo_server', 'ilhavo_testes', 'lousa', 'lousa3', 'lousa4', 'mhk', 'notaveis', 'obidos', 'oguest', 'rhv', 'santaclara', 'santajusta', 'sister', 'soure', 'soure_edit', 'soure_editor', 'soure_mfg', 'soure_public', 'soure_server', 'testes', 'toliveira', 'toliveira_reviewed', 'ucprosop']


## Connect to database

### Choose the database

In [17]:
db = 'soure'
connection_string = get_connection_string(db)
engine = get_engine(db)

## Database status

In [18]:
with engine.connect() as connection:
    classes = connection.execute(
        text('select class,count(*) as n from entities group by class')
    )
    classes
for c in classes:
    print(c['class'],c.n)

act 9123
acta 83
attribute 50335
class 18
geoentity 1783
group_element 10678
item 5
object 2
person 41951
relation 122623
rperson 442
source 158


## Map MHK database

In [19]:
from sqlalchemy import Column,String,Integer,DateTime, ForeignKey
from sqlalchemy.orm import declarative_base, relationship

Base = declarative_base()


## Entities
     
        |-----------|--------------|------|-----|---------------------+-----------------------------+
        | Field     | Type         | Null | Key | Default             | Extra                       |
        +-----------+--------------+------+-----+---------------------+-----------------------------+
        | id        | varchar(64)  | NO   | PRI | NULL                |                             |
        | class     | varchar(64)  | YES  | MUL | NULL                |                             |
        | inside    | varchar(64)  | YES  | MUL | NULL                |                             |
        | the_order | decimal(6,0) | YES  |     | NULL                |                             |
        | the_level | decimal(6,0) | YES  |     | NULL                |                             |
        | the_line  | decimal(6,0) | YES  |     | NULL                |                             |
        | groupname | varchar(32)  | YES  | MUL | NULL                |                             |
        | updated   | timestamp    | NO   | MUL | CURRENT_TIMESTAMP   | on update CURRENT_TIMESTAMP |
        | indexed   | timestamp    | NO   | MUL | 1974-04-25 00:00:01 |                             |
        +-----------+--------------+------+-----+---------------------+-----------------------------+

In [20]:

class Entity(Base):
    __tablename__ = 'entities'

    id = Column(String, primary_key=True)
    pom_class = Column('class',String)
    inside = Column(String)
    the_order = Column(Integer)
    the_level = Column(Integer)
    the_line = Column(Integer)
    groupname = Column(String)
    updated = Column(DateTime)
    indexed = Column(DateTime)

    # see https://docs.sqlalchemy.org/en/14/orm/inheritance.html
    __mapper_args__ = {
        'polymorphic_identity': 'entity',
        'polymorphic_on':pom_class
    }

    # untested
    @classmethod  # untested
    def get_subclasses(cls):
        for subclass in cls.__subclasses__():
            yield from subclass.get_subclasses()
            yield subclass
        
    def __repr__(self):
        return f'{self.groupname}${self.id}/type={self.pom_class}'


## Attributes
    +-----------+----------------+------+-----+---------+-------+
    | Field     | Type           | Null | Key | Default | Extra |
    +-----------+----------------+------+-----+---------+-------+
    | id        | varchar(64)    | NO   | PRI | NULL    |       |
    | entity    | varchar(64)    | YES  | MUL | NULL    |       |
    | the_type  | varchar(512)   | YES  | MUL | NULL    |       |
    | the_value | varchar(1024)  | YES  | MUL | NULL    |       |
    | the_date  | varchar(24)    | YES  | MUL | NULL    |       |
    | obs       | varchar(16000) | YES  |     | NULL    |       |
    +-----------+----------------+------+-----+---------+-------+

In [21]:

class Attribute(Entity):
    __tablename__ = 'attributes'

    id = Column(String, primary_key=True)
    entity = Column(String,ForeignKey('entities.id'))
    the_type = Column(String)
    the_value = Column(String)
    the_date = Column(String)
    obs = Column(String)

    the_entity = relationship("Entity", back_populates="attributes",)

    __mapper_args__ = {
        'polymorphic_identity':'attribute'
    }

    def __repr__(self):
        r = f'     ls${self.the_type}/{self.the_value}/{self.the_date}'
        if self.obs is not None:
                r = (f'{r}/obs={self.obs}')
        return r


Entity.attributes = relationship("Attribute", back_populates="the_entity")


/var/folders/sx/x6pyh9097n98mlz29_6njdj00000gn/T/ipykernel_76170/2845532560.py:1: SAWarning: Implicitly combining column entities.id with column attributes.id under attribute 'id'.  Please configure one or more attributes for these same-named columns explicitly.
  class Attribute(Entity):


## Relations   
    +-------------+----------------+------+-----+---------+-------+
    | Field       | Type           | Null | Key | Default | Extra |
    +-------------+----------------+------+-----+---------+-------+
    | id          | varchar(64)    | NO   | PRI | NULL    |       |
    | origin      | varchar(64)    | YES  | MUL | NULL    |       |
    | destination | varchar(64)    | YES  | MUL | NULL    |       |
    | the_date    | varchar(24)    | YES  | MUL | NULL    |       |
    | the_type    | varchar(32)    | YES  | MUL | NULL    |       |
    | the_value   | varchar(256)   | YES  | MUL | NULL    |       |
    | obs         | varchar(16000) | YES  |     | NULL    |       |
    +-------------+----------------+------+-----+---------+-------+

In [22]:

class Relation(Base):  # should extend Entity but gives error
    __tablename__ = 'relations'

    id = Column(String, primary_key=True)
    origin = Column(String,ForeignKey('entities.id'))
    org = relationship("Entity",foreign_keys=[origin], back_populates='rels_out')
    destination = Column(String,ForeignKey('entities.id'))
    dest = relationship("Entity",foreign_keys=[destination], back_populates="rels_in")
    the_type = Column(String)
    the_value = Column(String)
    the_date = Column(String)
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'relation'
    }

    def __repr__(self):
        r = f'     rel${self.the_type}/{self.the_value}/{self.destination}/{self.the_date}'
        if self.obs is not None:
                r = (f'{r}/obs={self.obs}')
        return r

Entity.rels_out = relationship("Relation",foreign_keys=[Relation.origin],back_populates="org")
Entity.rels_in = relationship("Relation",foreign_keys=[Relation.destination],back_populates="dest")


## Persons    
    +-------+----------------+------+-----+---------+-------+
    | Field | Type           | Null | Key | Default | Extra |
    +-------+----------------+------+-----+---------+-------+
    | id    | varchar(64)    | NO   | PRI | NULL    |       |
    | name  | varchar(128)   | YES  | MUL | NULL    |       |
    | sex   | char(1)        | YES  |     | NULL    |       |
    | obs   | varchar(16000) | YES  |     | NULL    |       |
    +-------+----------------+------+-----+---------+-------+

In [ ]:
class Person(Entity):
    __tablename__ = 'persons'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    name = Column(String)
    sex = Column(String(1))
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'person'
    }

## Objects

    +----------+----------------+------+-----+---------+-------+
    | Field    | Type           | Null | Key | Default | Extra |
    +----------+----------------+------+-----+---------+-------+
    | id       | varchar(64)    | NO   | PRI | NULL    |       |
    | name     | varchar(64)    | YES  | MUL | NULL    |       |
    | the_type | varchar(32)    | YES  |     | NULL    |       |
    | obs      | varchar(16000) | YES  |     | NULL    |       |
    +----------+----------------+------+-----+---------+-------+

In [ ]:
class Object(Entity):
    __tablename__ = 'objects'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    name = Column(String)
    the_type = Column(String(32))
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'object'
    }

## Acts

    +----------+----------------+------+-----+---------+-------+
    | Field    | Type           | Null | Key | Default | Extra |
    +----------+----------------+------+-----+---------+-------+
    | id       | varchar(64)    | NO   | PRI | NULL    |       |
    | the_type | varchar(32)    | YES  |     | NULL    |       |
    | the_date | varchar(24)    | YES  | MUL | NULL    |       |
    | loc      | varchar(64)    | YES  |     | NULL    |       |
    | ref      | varchar(64)    | YES  |     | NULL    |       |
    | obs      | varchar(16000) | YES  |     | NULL    |       |
    +----------+----------------+------+-----+---------+-------+


In [ ]:
class Act(Entity):
    __tablename__ = 'acts'

    id = Column(String, ForeignKey('entities.id'), primary_key=True)
    name = Column(String)
    the_type = Column(String(32))
    obs = Column(String)

    __mapper_args__ = {
        'polymorphic_identity':'act'
    }

/var/folders/sx/x6pyh9097n98mlz29_6njdj00000gn/T/ipykernel_75810/670107530.py:1: SAWarning: Reassigning polymorphic association for identity 'object' from <Mapper at 0x10808e160; Object> to <Mapper at 0x107fa20a0; Act>: Check for duplicate use of 'object' as value for polymorphic_identity.
  class Act(Entity):


## Sources

    +-----------+----------------+------+-----+---------+-------+
    | Field     | Type           | Null | Key | Default | Extra |
    +-----------+----------------+------+-----+---------+-------+
    | id        | varchar(64)    | NO   | PRI | NULL    |       |
    | the_type  | varchar(32)    | YES  |     | NULL    |       |
    | the_date  | varchar(24)    | YES  |     | NULL    |       |
    | loc       | varchar(64)    | YES  |     | NULL    |       |
    | ref       | varchar(64)    | YES  |     | NULL    |       |
    | kleiofile | varchar(1024)  | YES  |     | NULL    |       |
    | replaces  | varchar(254)   | YES  |     | NULL    |       |
    | obs       | varchar(16000) | YES  |     | NULL    |       |
    +-----------+----------------+------+-----+---------+-------+

## Classes

    +------------+-------------+------+-----+---------+-------+
    | Field      | Type        | Null | Key | Default | Extra |
    +------------+-------------+------+-----+---------+-------+
    | id         | varchar(64) | NO   | PRI | NULL    |       |
    | table_name | varchar(32) | YES  |     | NULL    |       |
    | group_name | varchar(32) | YES  |     | NULL    |       |
    | super      | varchar(64) | YES  |     | NULL    |       |
    +------------+-------------+------+-----+---------+-------+

## Class attributes

    +--------------+--------------+------+-----+---------+-------+
    | Field        | Type         | Null | Key | Default | Extra |
    +--------------+--------------+------+-----+---------+-------+
    | the_class    | varchar(64)  | YES  | MUL | NULL    |       |
    | name         | varchar(32)  | YES  |     | NULL    |       |
    | colname      | varchar(32)  | YES  |     | NULL    |       |
    | colclass     | varchar(32)  | YES  |     | NULL    |       |
    | coltype      | varchar(32)  | YES  |     | NULL    |       |
    | colsize      | decimal(5,0) | YES  |     | NULL    |       |
    | colprecision | decimal(6,0) | YES  |     | NULL    |       |
    | pkey         | decimal(1,0) | YES  |     | NULL    |       |
    +--------------+--------------+------+-----+---------+-------+


## Testing

In [ ]:
from sqlalchemy import select
from sqlalchemy.orm import Session

stmt = select(Entity).where(Entity.pom_class.not_in(['attribute','relation'])).limit(5)
print(stmt)
with Session(engine) as session:
    for row in session.execute(stmt):
        print(row)
    

In [65]:
print(Entity.__subclasses__())
for aclass in Entity.__subclasses__():
    print(aclass.__tablename__)

[<class '__main__.Person'>]
persons


In [63]:
from sqlalchemy import select
from sqlalchemy.orm import Session

stmt = select(Person).where(Person.id == '140625')
print(stmt)
with Session(engine) as session:
    for row in session.execute(stmt):
        entity: Entity=row[0]
        print(f'{entity.groupname}${entity.name}/id={entity.id}#{entity.pom_class}')
        for attribute in entity.attributes:
            print(f'     ls${attribute.the_type}/{attribute.the_value}/{attribute.the_date}',end='')
            if attribute.obs is not None:
                print(f'/obs={attribute.obs}')
            else:
                print()
        if len(entity.rels_out)>0:
            for rel in entity.rels_out:
                print(f'>{rel}')
        if len(entity.rels_in)>0:
            for rel in entity.rels_in:
                    print(rel,'<')               


SELECT entities.class, persons.id, entities.id AS id_1, entities.inside, entities.the_order, entities.the_level, entities.the_line, entities.groupname, entities.updated, entities.indexed, persons.name, persons.sex, persons.obs 
FROM entities JOIN persons ON entities.id = persons.id 
WHERE persons.id = :id_2
n$Arnaldo Henriques de Abreu/id=140625#person
     ls$código-de-referência/"PT/AUC/ELU/UC-AUC/B/001-001/A/000336"/20200211
     ls$data-do-registo/2018-04-19 14:33:58/20200211
     ls$url/"https://pesquisa.auc.uc.pt/details?id=140625"/20200211
     ls$uc.entrada/1835-10-17/1835-10-17
     ls$uc.saida/1838-10-09/1838-10-09
     ls$uc.entrada.ano/1835/1835-10-17
     ls$uc.saida.ano/1838/1838-10-09
     ls$nome/Arnaldo Henriques de Abreu/1835-10-17
     ls$nome.primeiro/Arnaldo/1835-10-17
     ls$nome.apelido/Henriques de Abreu/1835-10-17
     ls$nome.apelido/Abreu/1835-10-17
     ls$naturalidade/Porto/1835-10-17
     ls$naturalidade.ano/Porto.1835/1835-10-17
     ls$nome-geografico/P

## Source files

In [37]:
from pathlib import Path

kleio_files = [f.stem for f in list(Path('../sources').rglob('*.cli'))]
print("Number of kleio_files:", len(kleio_files))

imported_sources = %sql select sources.id, sources.kleiofile, entities.updated from sources left join entities on (sources.id=entities.id) order by updated desc 
sources_in_db = [s.id for s in imported_sources]
print("Number of imported files:",len(sources_in_db))

print("Files not in the database:", set(kleio_files)-set(sources_in_db))
print("Imported sources no file found:", set(sources_in_db)-set(kleio_files))



Number of kleio_files: 235
235 rows affected.
Number of imported files: 235
Files not in the database: set()
Imported sources no file found: set()


## Analyse attributes extracted from records

### Attributes in the database

In [8]:
nml = %sql select the_type, count(*) as tot from attributes group by the_type
for the_type, tot in nml:
    print(f'{tot:6} | {the_type}')

28 rows affected.
105300 | código-de-referência
   275 | colegio
105300 | data-do-registo
   186 | ec
 53627 | exame
 94164 | faculdade
 94164 | faculdade.ano
106372 | grau
106372 | grau.ano
319291 | matricula
319291 | matricula.ano
 98904 | naturalidade
 98904 | naturalidade.ano
105300 | nome
107486 | nome-geografico
101392 | nome-geografico.ano
198660 | nome.apelido
105300 | nome.primeiro
  8155 | nome.vide
  9975 | nota
  3142 | ordem-religiosa
  7036 | padre
  4170 | titulo
105300 | uc.entrada
 99014 | uc.entrada.ano
105300 | uc.saida
 99281 | uc.saida.ano
105300 | url


In [11]:
p1540 = %sql select id, name, pobs from nattributes where the_type='uc.entrada' and the_value like '1540%'
for id, name, pobs in p1540:
    print(f'{id:5} | {name} \n {pobs}\n\n')

473 rows affected.


ValueError: Unknown format code 'r' for object of type 'str'